In [1]:
import os
%pwd
os.chdir("../")
%pwd

'e:\\Deep Learning\\TENSORFLOW\\rice_image_detection'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from src.RICE_IMAGE_DETECTION.constants import *
from src.RICE_IMAGE_DETECTION.utils.common import read_yaml, create_directories

In [4]:
class ConfigureationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_cofig = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_cofig

In [5]:
import os
import zipfile
import gdown
from src.RICE_IMAGE_DETECTION import logger
from src.RICE_IMAGE_DETECTION.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self)->str:
        try:
            data_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {data_url} into file {zip_download_dir}")

            file_id = data_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {data_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def unzip_data(self):
        unzip_path = self.config.unzip_dir
        zip_download_dir = self.config.local_data_file
        os.makedirs(unzip_path, exist_ok=True)
        logger.info(f"unzipping data from {zip_download_dir} into file {unzip_path}")
        with zipfile.ZipFile(zip_download_dir, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"unzipping completed data from {zip_download_dir} into file {unzip_path}")

In [7]:
try:
    config = ConfigureationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    raise e

[2024-06-29 12:09:26,733: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-29 12:09:26,737: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-29 12:09:26,740: INFO: common: created directory at: artifacts]
[2024-06-29 12:09:26,742: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-29 12:09:26,743: INFO: 1702304497: Downloading data from https://drive.google.com/file/d/1Qqb8zsCSUecbW8sSRzfQ37WJgEAaDPdF/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1Qqb8zsCSUecbW8sSRzfQ37WJgEAaDPdF
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1Qqb8zsCSUecbW8sSRzfQ37WJgEAaDPdF&confirm=t&uuid=68637090-55ce-4d6e-8488-8647992dd1bf
To: e:\Deep Learning\TENSORFLOW\rice_image_detection\artifacts\data_ingestion\data.zip
100%|██████████| 229M/229M [00:45<00:00, 4.99MB/s] 


[2024-06-29 12:10:14,849: INFO: 1702304497: Downloaded data from https://drive.google.com/file/d/1Qqb8zsCSUecbW8sSRzfQ37WJgEAaDPdF/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-06-29 12:10:14,849: INFO: 1702304497: unzipping data from artifacts/data_ingestion/data.zip into file artifacts/data_ingestion]
[2024-06-29 12:11:36,022: INFO: 1702304497: unzipping completed data from artifacts/data_ingestion/data.zip into file artifacts/data_ingestion]
